In [42]:
from bs4 import BeautifulSoup
import time
from random import randint
import pandas as pd
import re
import sqlite3
import numpy as np
import spacy
from gensim import corpora
from collections import defaultdict

In [5]:
conn = sqlite3.connect('lyrics.db')

In [6]:
nlp = spacy.load('en')

## Clean Lyrics Database

In [4]:
records = pd.read_sql_query("select * from lyrics_abc where valid = 1;",conn)

In [35]:
# allrecords = pd.read_sql_query("select * from lyrics_abc;",conn)
# allrecords['duplicate'] = allrecords['url'].duplicated()

# def update(x):
#     if(x['duplicate']==True)&(x['status']==0):
#         return 1
#     elif x['status'] == 1 : 
#         return 1
#     else : return 0

# allrecords['new_status'] = allrecords.apply(lambda x: update(x),axis=1)
# allrecords['status'] = allrecords['new_status']
# del allrecords['new_status']

# allrecords[allrecords['url']=='http://cyberhymnal.org/htm/b/e/beautxms.htm']
# print('scrapped:',allrecords[allrecords['status']==1]['status'].count())
# print('not scrapped:',allrecords[allrecords['status']==0]['status'].count())
# print('total:',allrecords['status'].count())
# del allrecords['duplicate']

# allrecords.to_sql('lyrics_abc',conn,if_exists='replace')

In [72]:
# def update(x):
#     if x['fullhtml'] is None:
#         return 0
#     elif x['song_name'] == "  Page not found - CyberHymnal" :
#         return 0
#     else :
#         return 1

# records[records['fullhtml'].notnull()]
# records.iloc[7692]['song_name']
# records['valid'] = records.apply(lambda x:update(x),axis=1)
# records.to_sql('lyrics_abc',conn,if_exists='replace')

In [75]:
columns = ['key','title','composer_music','music_year','composer_lyrics','lyrics_year','lyrics']
df = pd.DataFrame(columns=columns)

for reckey,record in records.iterrows():
    soup = BeautifulSoup(record['fullhtml'],"lxml")
    key = reckey
    title = soup.title.get_text()
    #print(title," ",key)
    
    composer = re.findall("Words:(.*) Music:(.*?)\"",record['meta'])
    if len(composer) > 0 :
        music = re.findall("[ ]?(.*?)[,.].*",composer[0][1])
        #music = re.findall("(.*)[,.].*",composer[0][1])
        music_yr = re.findall(".*(\d{4})",composer[0][1])
        composer_music = music[0] if len(music) > 0 else 0
        music_year = music_yr[0] if len(music_yr) > 0 else 0
        #print("Music:",composer_music)
        #print("Year:",music_year)


        words = re.findall("[ ]?(.*?),.*",composer[0][0])
        words_year = re.findall(".*(\d{4})",composer[0][0])   

        composer_lyrics = words[0] if len(words) > 0 else 0
        lyrics_year = words_year[0] if len(words_year) > 0 else 0
        #print("Lyrics:",composer_lyrics)
        #print("Year:",lyrics_year)
    else : 
        composer_lyrics = ""
        lyrics_year = 0 
        composer_music = ""
        music_year = 0 
          
    lyrics = BeautifulSoup(record['lyrics'],"lxml").get_text()
    #print(lyrics)
    
    row = [key,title,composer_music,music_year,composer_lyrics,lyrics_year,lyrics]
    #print(composer)
    #print([composer_music,music_year,composer_lyrics,lyrics_year])
    df.loc[len(df)] = row 
    
    
    #print('--------------------------')
df.to_sql('cleaned_lyrics',conn,if_exists='replace')

## Analyze Database

In [7]:
records = pd.read_sql_query("select * from cleaned_lyrics;",conn)

In [ ]:
# TODO : Explore Gensim Phrases

In [30]:
documents = []
# x = 10 
for key,row in records.iterrows():
    clean_lyrics = row['lyrics'].replace('\r\n',' ').replace('\n',' ')
    processed_lyrics = nlp(clean_lyrics)
    token_lyrics = [token.text for token in processed_lyrics if token.is_stop != True and token.is_punct != True and str(token) != ' ']
    if len(token_lyrics) > 0 :documents.append(token_lyrics)
    #debug
    #x = x - 1 
    #if x == 0 : break
dictionary = corpora.Dictionary(documents)    
dictionary.save('hymnal.dict')
print(dictionary)

Dictionary(17207 unique tokens: ['confounded', 'chalices', 'benumb', 'sepulcher', 'wrecked']...)


In [38]:
for k,v in dictionary.items():
    print(k," ",v)

4221   confounded
16764   chalices
11338   benumb
8115   sepulcher
4059   wrecked
10962   behalf
12764   pacific
16178   endris
2642   drops
11266   sayeth
1186   glorious
7999   nightly
8329   sheepfold
10678   starlit
460   faithless
5154   solely
11452   Shamed
13374   instrument
4147   abides
16541   loosen
6545   harvesting
4012   hastening
13127   blights
17045   Bashan
14262   unwary
14573   wistful
1430   forth
11935   brutes
8188   Suffice
8300   clanging
13872   brazen
3239   wider
7678   wending
13273   quill
3609   undisturbed
3836   Shake
693   rapture
11297   shocks
15167   sickening
8845   o’erpower
13382   Torrents
13368   Spice
12946   cov’ring
1486   snares
4349   Isle
12246   cluster
3048   fathers
6725   Meek
4390   sparkling
13238   wives
1304   Scarce
430   sing
14756   protest
4447   caroling
10972   thoughtfully
4726   valor
15125   besiege
6663   receivèd
11003   certified
16983   Lullabies
15253   Summoned
2680   Emmanuel
8731   unruly
2523   towers
3593   cou

In [40]:
corpus = [dictionary.doc2bow(text) for text in documents]
corpora.MmCorpus.serialize('hymnal.mm',corpus)

In [43]:
frequency = defaultdict(int)
for text in documents:
    for token in text:
        frequency[token] += 1

In [47]:
import pickle

with open('frequency.txt', 'wb') as fp:
    pickle.dump(frequency, fp)

In [56]:
frequency['confounded']
ordered = sorted(frequency.items(),key=lambda k:k[1],reverse=True)

In [59]:
ordered[1:20]

[('’s', 7351),
 ('Refrain', 7334),
 ('God', 6805),
 ('Lord', 6463),
 ('O', 5958),
 ('Thee', 5520),
 ('shall', 4970),
 ('love', 4893),
 ('Jesus', 4750),
 ('Thou', 4481),
 ('day', 2816),
 ('heart', 2739),
 ('Savior', 2468),
 ('life', 2449),
 ('Christ', 2265),
 ('grace', 2248),
 ('glory', 2176),
 ('soul', 2096),
 ('praise', 2061)]

In [31]:
# mapping between words and id
print(dictionary.token2id)

{'confounded': 4221, 'chalices': 16764, 'benumb': 11338, 'sepulcher': 8115, 'wrecked': 4059, 'behalf': 10962, 'pacific': 12764, 'endris': 16178, 'drops': 2642, 'sayeth': 11266, 'glorious': 1186, 'nightly': 7999, 'sheepfold': 8329, 'starlit': 10678, 'faithless': 460, 'solely': 5154, 'Shamed': 11452, 'instrument': 13374, 'abides': 4147, 'loosen': 16541, 'harvesting': 6545, 'hastening': 4012, 'blights': 13127, 'Bashan': 17045, 'unwary': 14262, 'wistful': 14573, 'forth': 1430, 'brutes': 11935, 'Suffice': 8188, 'clanging': 8300, 'brazen': 13872, 'wider': 3239, 'wending': 7678, 'quill': 13273, 'undisturbed': 3609, 'Shake': 3836, 'rapture': 693, 'shocks': 11297, 'sickening': 15167, 'o’erpower': 8845, 'Torrents': 13382, 'Spice': 13368, 'cov’ring': 12946, 'snares': 1486, 'Isle': 4349, 'cluster': 12246, 'fathers': 3048, 'Meek': 6725, 'sparkling': 4390, 'wives': 13238, 'Scarce': 1304, 'sing': 430, 'protest': 14756, 'caroling': 4447, 'thoughtfully': 10972, 'valor': 4726, 'besiege': 15125, 'receivè

In [21]:
lyrics = records['lyrics'][0]
print(lyrics)
clean_lyrics = lyrics.replace('\r\n',' ').replace('\n',' ')
print(clean_lyrics)


“Abba, Father!” We approach Thee
In our Savior’s precious Name;
We, Thy children, here assembled,
Now Thy promised blessing claim;
From our sins His blood hath washed us,
’Tis through Him our souls draw nigh,
And Thy Spirit, too, hath taught us,
“Abba, Father,” thus to cry.
Once as prodigals we wandered
In our folly far from Thee,
But Thy grace, o’er sin abounding,
Rescued us from misery;
Thou Thy prodigals hast pardoned,
Kissed us with a Father’s love,
Spread the festive board, and called us,
E’er to dwell with Thee above.
Clothed in garments of salvation,
At Thy table is our place,
We rejoice, and Thou rejoicest,
In the riches of Thy grace;
“It is meet,” we hear Thee saying,
“We should merry and be glad,
I have found My once lost children,
Now they live who once were dead.”
“Abba, Father!” all adore Thee,
All rejoice in Heav’n above,
While in us they learn the wonders
Of Thy wisdom, grace, and love;
Soon before Thy throne assembled,
All Thy children shall proclaim,
“Glory, everlasti

In [22]:
slyrics = nlp(lyrics)
sclyrics = nlp(clean_lyrics)

swords = [token.text for token in slyrics if token.is_stop != True and token.is_punct != True]
scwords = [token.text for token in sclyrics if token.is_stop != True and token.is_punct != True]
# words = [token.text for token in lyrics_1 if token.is_stop != True and token.is_punc != True and token.pos_ == 'NOUN']
print(swords)
print("------------------")
print(scwords)

# for num,sentence in enumerate(lyrics_1.sents):
#     print(num," ",sentence)


['\n', 'Abba', 'Father', 'approach', 'Thee', '\r\n', 'Savior', '’s', 'precious', '\r\n', 'Thy', 'children', 'assembled', '\r\n', 'Thy', 'promised', 'blessing', 'claim', '\r\n', 'sins', 'blood', 'hath', 'washed', '\r\n', '’Tis', 'souls', 'draw', 'nigh', '\r\n', 'Thy', 'Spirit', 'hath', 'taught', '\r\n', 'Abba', 'Father', 'cry', '\n', 'prodigals', 'wandered', '\r\n', 'folly', 'far', 'Thee', '\r\n', 'Thy', 'grace', 'o’er', 'sin', 'abounding', '\r\n', 'Rescued', 'misery', '\r\n', 'Thou', 'Thy', 'prodigals', 'hast', 'pardoned', '\r\n', 'Kissed', 'Father', '’s', 'love', '\r\n', 'Spread', 'festive', 'board', 'called', '\r\n', 'E’er', 'dwell', 'Thee', '\n', 'Clothed', 'garments', 'salvation', '\r\n', 'Thy', 'table', 'place', '\r\n', 'rejoice', 'Thou', 'rejoicest', '\r\n', 'riches', 'Thy', 'grace', '\r\n', 'meet', 'hear', 'Thee', 'saying', '\r\n', 'merry', 'glad', '\r\n', 'found', 'lost', 'children', '\r\n', 'live', 'dead', '\n', 'Abba', 'Father', 'adore', 'Thee', '\r\n', 'rejoice', 'Heav’n', '

In [ ]:
## Clean the Reviews.

In [ ]:
## Perform LSTM 
# https://nicschrading.com/project/Intro-to-NLP-with-spaCy/
# http://blog.sharepointexperience.com/2016/01/nlp-and-sharepoint-part-1/
# https://github.com/JonathanReeve/advanced-text-analysis-workshop-2017/blob/master/advanced-text-analysis.ipynb
# http://nbviewer.jupyter.org/github/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb